In [1]:
cd c:\Users\bonje\Documents\Perso\ploutos


c:\Users\bonje\Documents\Perso\ploutos


c:\Users\bonje\AppData\Local\pypoetry\Cache\virtualenvs\ploutos-wcQ0yzvx-py3.11\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:

from pandas import Timestamp

from dotenv import load_dotenv
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from ploutos.utils.sheet_migration import SheetMigration
load_dotenv()

import sys
from importlib import reload
from ploutos.api.bank_api import BankApi, Bank
if 'ploutos.api.bank_api' in sys.modules:
    reload(sys.modules['ploutos.api.bank_api'])
else:
    print("BankApi module not loaded yet - no need to reload")


In [3]:
from ploutos.api.bank_api import BankApi, Bank
account_api=BankApi(Bank.LCL)
sheet_migration=SheetMigration()

print(account_api.get_metadata())

Bank.LCL <enum 'Bank'> True
Successfully connected to Bank.LCL
{'id': '1bdb6174-5522-44d5-a67e-28d50123966f', 'created': '2024-06-16T16:54:38.946472Z', 'last_accessed': '2024-12-26T07:40:11.850067Z', 'iban': 'FR2930002010170000082489B97', 'institution_id': 'LCL_CRLYFRPP', 'status': 'READY', 'owner_name': '', 'bban': None}


In [4]:
print(sheet_migration.account_list)


               Compte  Fin Janvier 2023       Actuel
0      Compte Courant           -760.35    -621,39 €
1        Livret Jeune             10.00   1 621,45 €
2               Lydia              4.05       8,41 €
3                Izly              7.60       0,00 €
4             Espèces              0.10       0,00 €
5            Tricount             23.21      38,84 €
6       Remboursement           -662.08       0,00 €
7      Lydia Vêtement              0.00       0,00 €
8          Lydia Vélo            510.43       0,00 €
9   Ticket Restaurant              0.00     156,18 €
10               Oddo           4431.26   2 931,26 €
11           Livret A              0.00  12 791,52 €
12  Carrefour Natixis              0.00   6 715,73 €
13            Revolut              0.00       0,00 €


In [5]:
df_paiement=sheet_migration.get_data("Paiement")
df_transfert=sheet_migration.get_data("Transfert")
df_benefices=sheet_migration.get_data("Bénéfices")


Date min: 2023-02-01 00:00:00
Date max: 2024-06-22 00:00:00
Nombre de lignes: 1310
Date min: 2023-02-03 00:00:00
Date max: 2024-06-16 00:00:00
Nombre de lignes: 113
Date min: 2023-02-01 00:00:00
Date max: 2024-06-05 00:00:00
Nombre de lignes: 158


### LCL

#### Compte Courant

In [6]:
from ploutos.utils.scraping.scrap_lcl import convert_LCLpdf_to_df

old_lcl=convert_LCLpdf_to_df("ploutos/data/lcl/compte_courant")

  0%|          | 0/4 [00:00<?, ?it/s]Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'
100%|██████████| 4/4 [00:11<00:00,  3.00s/it]
4it [00:00, 3813.87it/s]


In [38]:
def calculate_last_transaction(db_payement, db_transfert, db_benefices,account):
    last_transaction=[]
    last_transaction.append(db_payement[db_payement["Compte"]==account]["Date"].max())
    last_transaction.append(db_transfert[db_transfert["Compte arrivée"]==account]["Date"].max())
    last_transaction.append(db_transfert[db_transfert["Compte départ"]==account]["Date"].max())
    last_transaction.append(db_benefices[db_benefices["Compte"]==account]["Date"].max())
    last_transaction=[ts for ts in last_transaction if pd.notna(ts)]
    if len(last_transaction)>0:
        last_transaction=max(last_transaction)
    else:
        last_transaction=None
    return last_transaction


In [39]:
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Compte Courant").strftime("%Y-%m-%d")
transactions=account_api.get_transactions(date_from='2024-06-06')
print(last_transaction)


2024-06-16


,transactionId,entryReference,Date,Date valeur,Montant_dict,Description,internalTransactionId,Montant,Type
0,c7c6fb8c-b433-4b6e-ae51-d26c1a3f0bae,c7c6fb8c-b433-4b6e-ae51-d26c1a3f0bae,2024-12-19,2024-12-19,"{'amount': '-3191.52', 'currency': 'EUR'}",PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA EPA...,54b07613266144b815fd4a2aa60c7927,-3191.52,Booked
1,9604ae60-eeae-4eb8-8053-20f71b758299,9604ae60-eeae-4eb8-8053-20f71b758299,2024-12-19,2024-12-19,"{'amount': '1000', 'currency': 'EUR'}",VIREMENT\n\nVIREMENT M JEREMY BON\n\n\n\nSCR43...,e2b373383d2baeeaff7283b08b15121a,1000.00,Booked


In [40]:
debit_old=old_lcl[(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['DEBIT'].sum()
credit_old=old_lcl[(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['CREDIT'].sum()
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Compte Courant"]["Actuel"].values[0].replace("€","").replace(",","."))
end_september_balance=float(sheet_value)+credit_old-debit_old
print(f"End of September balance: {end_september_balance}")
current_balance=end_september_balance+transactions[transactions["Date"]>'2024-09-30']['Montant'].sum()
print(f"Current balance: {current_balance}")
official_balance=account_api.get_balance()
print(f"Official balance: {official_balance}")


End of September balance: 2032.9199999999983
Current balance: 407.3699999999976
[{'balanceAmount': {'amount': '530.82', 'currency': 'EUR'}, 'balanceType': 'expected', 'referenceDate': '2024-12-23'}, {'balanceAmount': {'amount': '407.37', 'currency': 'EUR'}, 'balanceType': 'closingBooked', 'referenceDate': '2024-12-20'}]
Official balance: 407.37


#### Compte Epargne


Le livret jeune n'a pas subi de changement et va fermer. On regarde uniquement le livret A.
On détecte les transactions de M JEREMY BON dans le compte courant avec `(transactions["Description"].str.contains("M JEREMY BON"))`

In [31]:
df_transfert[df_transfert["Compte départ"]=="Livret A"]

,Identifiant,Date,Montant,Type,Description,Compte départ,Compte arrivée
95,23TRDÉCE07,2023-12-28,5000.0,Investissement,natixis carrefour,Livret A,Carrefour Natixis
109,24TRJUIN02,2024-06-16,1691.9,Transfert interne,transfert caf + loyer,Livret A,Compte Courant
110,24TRJUIN03,2024-06-16,5700.0,Transfert interne,avril+mai+juin16,Livret A,Compte Courant


In [42]:
convert_LCLpdf_to_df("ploutos/data/lcl/compte_livret")
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Livret A")
print(last_transaction)


100%|██████████| 1/1 [00:05<00:00,  5.93s/it]
1it [00:00, 124.95it/s]

2024-06-16 00:00:00


In [65]:
debit_old=old_lcl[(old_lcl["Description"].str.contains("M JEREMY BON"))&(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['DEBIT'].sum()
credit_old=old_lcl[(old_lcl["Description"].str.contains("M JEREMY BON"))&(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['CREDIT'].sum()
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Livret A"]["Actuel"].values[0].replace("€","").replace(",",".").replace("\u202f",""))
end_september_balance=float(sheet_value)-credit_old+debit_old
print(f"End of September balance: {end_september_balance}")
current_balance=end_september_balance-transactions[(transactions["Description"].str.contains("M JEREMY BON"))&(transactions["Date"]>'2024-09-30')]['Montant'].sum()
print(f"Current balance: {current_balance}")
official_balance=8000
print(f"Official balance: {official_balance}")

End of September balance: 10591.52
Current balance: 8000.0
Official balance: 8000


### Lydia

In [70]:
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Lydia")
print(last_transaction)
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Lydia"]["Actuel"].values[0].replace("€","").replace(",",".").replace("\u202f",""))
print(f"Lydia balance on {last_transaction.strftime('%Y-%m-%d')}: {sheet_value}")


2024-06-22 00:00:00
Lydia balance on 2024-06-22: 8.41


In [79]:
lydia_account_value=50.84
new_transaction_lydia=pd.DataFrame([{
    "Date":"2024-09-01",
    "Libellé":"Madrid Gael Terier",
    "Montant":38.89,
    "Compte":"Lydia",
},
{
    "Date":"2024-08-31",
    "Libellé":"Guilhem Leborgne",
    "Montant": -4,
    "Compte":"Lydia",
    
},
{
    "Date":"2024-08-27",
    "Libellé":"Dépôt crte revolut 7255",
    "Montant": 25,
    "Compte":"Lydia",
},{
    "Date":"2024-08-27",
    "Libellé":"Guilhem Leborgne",
    "Montant": -25,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-25",
    "Libellé":"Kdo Juan Lucie Formery",
    "Montant": -8.33,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-25",
    "Libellé":"Juanita PA",
    "Montant": -3.33,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-16",
    "Libellé":"Tren Gijon Lucie Formery",
    "Montant": 27.1,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-01",
    "Libellé":"Thai Bastien Englebert",
    "Montant": -7.9,
    "Compte":"Lydia",
}
])
assert new_transaction_lydia['Montant'].sum()+sheet_value==lydia_account_value



### Revolut


In [1]:
cd c:\Users\bonje\Documents\Perso\ploutos


c:\Users\bonje\Documents\Perso\ploutos


c:\Users\bonje\AppData\Local\pypoetry\Cache\virtualenvs\ploutos-wcQ0yzvx-py3.11\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:

from ploutos.api.bank_api import BankApi, Bank


In [5]:
lcl_api.get_transactions()

,transactionId,entryReference,Date,Date valeur,Montant_dict,Description,internalTransactionId,Montant,Type
0,19bbbc83-27c4-48ec-81b8-f107331284aa,19bbbc83-27c4-48ec-81b8-f107331284aa,2024-12-24,2024-12-24,"{'amount': '-186.88', 'currency': 'EUR'}",PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA CAR...,eabb272d4c79ec85213d4937265e01a3,-186.88,Booked
1,21b35710-9549-4d1a-8c9f-98aefa8cd908,21b35710-9549-4d1a-8c9f-98aefa8cd908,2024-12-23,2024-12-23,"{'amount': '-234.44', 'currency': 'EUR'}",PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA CAR...,8dc28dfacbcf10bb7cc08644449e514b,-234.44,Booked
2,cdc8c21c-c916-4f75-8cef-4ce39eb824b4,cdc8c21c-c916-4f75-8cef-4ce39eb824b4,2024-12-21,2024-12-21,"{'amount': '858.89', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST PAPEREUX VERON...,5ae676f9941220df5f4505874fc4c181,858.89,Booked
3,67223268-c5bd-41db-8c91-9f59cf298ccb,67223268-c5bd-41db-8c91-9f59cf298ccb,2024-12-20,2024-12-20,"{'amount': '-1', 'currency': 'EUR'}",CHANGE A TERME-COM.CHANGE VENTE\n\nFRAIS VIR I...,e94f653b2fb471136b73170853a8d7d8,-1.00,Booked
4,338a8d1b-6e12-4e13-9471-f0c1d5fac049,338a8d1b-6e12-4e13-9471-f0c1d5fac049,2024-12-20,2024-12-20,"{'amount': '-500', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST Jeremy Bon rev...,6eaf07cecc5c426e00aded5c611e4b5e,-500.00,Booked
...,...,...,...,...,...,...,...,...,...
56,33856a27-385e-46af-9dc8-12ccda507cb8,33856a27-385e-46af-9dc8-12ccda507cb8,2024-10-07,2024-10-07,"{'amount': '-250', 'currency': 'EUR'}",PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA EPA...,ea53ba70844b537f0e9ada5a90c60a9b,-250.00,Booked
57,db5f44d2-8921-4bd6-8261-22bdf23e1895,db5f44d2-8921-4bd6-8261-22bdf23e1895,2024-10-02,2024-10-02,"{'amount': '100', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST MME PAPEREUX N...,1a4a43966a1eb1e29758b7db7dcc0dc9,100.00,Booked
58,d52ed06d-5e4f-4394-83ce-06b63c5bbbd6,d52ed06d-5e4f-4394-83ce-06b63c5bbbd6,2024-09-30,2024-09-30,"{'amount': '710', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST Adrien BRUN\nL...,dff023ef479eeb8a17a95f54f175ce4a,710.00,Booked
59,ad6bbc08-1e2c-481b-9f2b-853f1d1608bf,ad6bbc08-1e2c-481b-9f2b-853f1d1608bf,2024-09-30,2024-09-30,"{'amount': '-668.55', 'currency': 'EUR'}",VIREMENT\n\nVIR SEPA vulpian nouveau\nbon vulp...,13dd5291cfb0c25f7974b30d3026bde3,-668.55,Booked


In [3]:
lcl_api=BankApi(Bank.LCL)
lcl_api.api.get_transactions()


Bank.LCL <enum 'Bank'> True
Successfully connected to Bank.LCL


{'transactions': {'booked': [{'transactionId': '19bbbc83-27c4-48ec-81b8-f107331284aa',
    'entryReference': '19bbbc83-27c4-48ec-81b8-f107331284aa',
    'bookingDate': '2024-12-24',
    'valueDate': '2024-12-24',
    'transactionAmount': {'amount': '-186.88', 'currency': 'EUR'},
    'remittanceInformationUnstructuredArray': ['PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA CARREFOUR BANQUE\nICS.FR83ZZZ135674\n.RUM.CS00-5135889367110\n0\nSDR435844993947'],
    'internalTransactionId': 'eabb272d4c79ec85213d4937265e01a3'},
   {'transactionId': '21b35710-9549-4d1a-8c9f-98aefa8cd908',
    'entryReference': '21b35710-9549-4d1a-8c9f-98aefa8cd908',
    'bookingDate': '2024-12-23',
    'valueDate': '2024-12-23',
    'transactionAmount': {'amount': '-234.44', 'currency': 'EUR'},
    'remittanceInformationUnstructuredArray': ['PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA CARREFOUR BANQUE\nICS.FR83ZZZ135674\n.RUM.CS00-5135889367110\n0\nSDR435543806434'],
    'internalTransactionId': '8dc28dfacbcf10bb7cc0

In [4]:
revolut_api=BankApi(Bank.REVOLUT)


Bank.REVOLUT <enum 'Bank'> True
Successfully connected to Bank.REVOLUT


In [12]:
raw_transac=revolut_api.api.get_transactions()

In [19]:
pd.DataFrame(raw_transac['transactions']['pending'])

,transactionId,bookingDate,bookingDateTime,transactionAmount,creditorName,remittanceInformationUnstructuredArray,proprietaryBankTransactionCode
0,676e38ac-43f9-a794-b149-4d48950e9181,2024-12-27,2024-12-27T05:18:36.737745Z,"{'amount': '-40.85', 'currency': 'EUR'}",FREE MOBILE,"[To Free Mobile, FM-58933087-2]",TRANSFER
1,675b0d4c-5a51-a0d8-81ba-3f3900affeff,2024-12-12,2024-12-12T16:20:28.704598Z,"{'amount': '-150.00', 'currency': 'EUR'}",Carrefourlocnet,[Carrefourlocnet],CARD_PAYMENT


In [5]:
transac_rev=revolut_api.get_transactions()


Date and Date valeur are not the same
Error with length of description (Max length is greater than 1)
Error with length of description (Max length is greater than 1)


### Autres

In [6]:
transac_rev

,transactionId,Date,Date valeur,bookingDateTime,valueDateTime,Montant_dict,creditorName,Description,proprietaryBankTransactionCode,internalTransactionId,currencyExchange,debtorName,debtorAccount,creditorAccount,additionalDataStructured,Montant,Type
0,676ddb69-43be-a811-b18e-122aaab8fd8f,2024-12-26,2024-12-27,2024-12-26T22:40:41.243875Z,2024-12-27T09:35:16.196501Z,"{'amount': '-6.99', 'currency': 'EUR'}",Amazon Prime Fr 2469664,Amazon Prime Fr,CARD_PAYMENT,a6801c8dbdb09c3edcb70447e37c7332,NaN,NaN,NaN,NaN,NaN,-6.99,Booked
1,6769b70c-a5ec-a08e-b7de-45cf559238ab,2024-12-23,2024-12-25,2024-12-23T19:16:28.587891Z,2024-12-25T11:13:21.836815Z,"{'amount': '-2.60', 'currency': 'EUR'}",Avia,Avia,CARD_PAYMENT,a964cff762b52304f3f8244d8d20f7af,NaN,NaN,NaN,NaN,NaN,-2.60,Booked
2,67695c44-8772-a9ef-89f5-a9528c09fb26,2024-12-23,2024-12-25,2024-12-23T12:49:08.804907Z,2024-12-25T09:17:32.550402Z,"{'amount': '-16.40', 'currency': 'EUR'}",Papa Jo,Papa Jo,CARD_PAYMENT,eff9a03e7e8a9a4d119034881e646415,NaN,NaN,NaN,NaN,NaN,-16.40,Booked
3,676957dc-d904-a978-9373-c09a0fff8917,2024-12-23,2024-12-25,2024-12-23T12:30:20.374217Z,2024-12-25T11:06:41.861609Z,"{'amount': '-32.79', 'currency': 'EUR'}",Relais Fnac 0389974,Relais Fnac,CARD_PAYMENT,a544c1f07cbd43c90c86d4e521e3a240,NaN,NaN,NaN,NaN,NaN,-32.79,Booked
4,676961a8-a7c4-a546-ae24-2b0af4510318,2024-12-23,2024-12-24,2024-12-23T13:12:08.060012Z,2024-12-24T11:10:32.051135Z,"{'amount': '-69.82', 'currency': 'EUR'}",Babbel.com 154931523,Babbel.com 154931523,CARD_PAYMENT,6f148e4cb75532095ab1b284fde0811d,NaN,NaN,NaN,NaN,NaN,-69.82,Booked
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,66f88e7a-d40d-accf-80ae-71f355054baa,2024-09-28,2024-09-30,2024-09-28T23:17:14.677925Z,2024-09-30T14:21:13.974435Z,"{'amount': '-12.72', 'currency': 'EUR'}",Ticketsbolivia.com,ticketsbolivia.com,CARD_PAYMENT,31b3aa7e93c3dd34cb92fffc59566b8d,"{'instructedAmount': {'amount': '12.48', 'curr...",NaN,NaN,NaN,"{'chargeAmount': {'amount': '0.24', 'currency'...",-12.72,Booked
390,66f87bda-aa26-ad6e-bac9-9beffe011ecb,2024-09-28,2024-09-30,2024-09-28T21:57:46.460762Z,2024-09-30T14:22:30.781078Z,"{'amount': '-4.95', 'currency': 'EUR'}",Culqi*neyra Collection,Culqi*neyra Collection,CARD_PAYMENT,28ffad4f7562a2248ac048461500ee2a,"{'instructedAmount': {'amount': '4.85', 'curre...",NaN,NaN,NaN,"{'chargeAmount': {'amount': '0.10', 'currency'...",-4.95,Booked
391,66f87d14-f997-ad7a-b84d-47b964d687ac,2024-09-28,2024-09-29,2024-09-28T22:03:00.155685Z,2024-09-29T12:55:42.452366Z,"{'amount': '-2.47', 'currency': 'EUR'}",Izi*artesanias Cusco,Izi*artesanias Cusco,CARD_PAYMENT,6a43c98206b89bb36e4efa6562f87a9e,"{'instructedAmount': {'amount': '2.43', 'curre...",NaN,NaN,NaN,"{'chargeAmount': {'amount': '0.04', 'currency'...",-2.47,Booked
0,676e38ac-43f9-a794-b149-4d48950e9181,2024-12-27,NaT,2024-12-27T05:18:36.737745Z,NaN,"{'amount': '-40.85', 'currency': 'EUR'}",FREE MOBILE,To Free Mobile,TRANSFER,NaN,NaN,NaN,NaN,NaN,NaN,-40.85,Pending


In [67]:
sheet_migration.account_list

,Compte,Fin Janvier 2023,Actuel
0,Compte Courant,-760.35,"-621,39 €"
1,Livret Jeune,10.00,"1 621,45 €"
2,Lydia,4.05,"8,41 €"
3,Izly,7.60,"0,00 €"
4,Espèces,0.10,"0,00 €"
5,Tricount,23.21,"38,84 €"
6,Remboursement,-662.08,"0,00 €"
7,Lydia Vêtement,0.00,"0,00 €"
8,Lydia Vélo,510.43,"0,00 €"
9,Ticket Restaurant,0.00,"156,18 €"


    ### Tricount (A faire)

### ticket resto (A solde)

#### Natixis Interépargne 
